# Computing photometry of Diffsky galaxies

This notebook demonstrates how to compute photometry through arbitrary bandpasses of a population of Diffsky galaxies. 

In [ ]:
# ! wget -q https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_fsps_v3.2_lgmet_age.sparse.h5

## Load the mock data

This cells show how to load the mock natively produced by the diffsky source code. For purposes of this demo, we will just work with a single lightcone patch of the natively-generated hdf5 files, which will be downloaded in the next cell.

#### Using the OpenCosmo toolkit
The hdf5 files produced by diffsky are later ingested by the [OpenCosmo](https://opencosmo.readthedocs.io/en/stable/) toolkit, which enables efficient querying, map-making, and other features. See [Accessing and Working With Diffsky Mock Galaxy Catalogs](https://github.com/ArgonneCPAC/opencosmo-examples/blob/main/03-Diffsky/demo_diffmah_diffstar.ipynb) for a tutorial on how to load diffsky mocks with OpenCosmo.

In [ ]:
import os
from diffsky.data_loaders import load_flat_hdf5

drn_mock = "/Users/aphearin/work/DATA/random_data/1122/smdpl_dr1"
basename_list = ["lc_cores-411.0.diffsky_gals.hdf5", ]
mock_collector = []
for bn in basename_list:
    fn_mock = os.path.join(drn_mock, bn)
    mock_this_bn = load_flat_hdf5(fn_mock, dataset='data')
    mock_collector.append(mock_this_bn)
    
mock = {key: np.concatenate([mock[key] for mock in mock_collector]) for key in mock_collector[0].keys()}

In [ ]:
from diffsky.data_loaders.hacc_utils import load_lc_mock as llcm

mock_info = llcm.load_lc_mock_info(fn_mock)
print(mock_info.keys())

In [ ]:
import recompute_phot

In [ ]:
ssp_mag_table, wave_eff_table = recompute_phot.get_precomputed_phot_info(mock_info)

In [ ]:
phot_data = recompute_phot.compute_mock_photometry(mock, mock_info, ssp_mag_table, wave_eff_table)
phot_data.keys()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
__=ax0.scatter(mock['lsst_u'], phot_data['obs_mags'][:, 0], s=1)
__=ax1.hist(phot_data['obs_mags'][:, 0]-mock['lsst_u'], alpha=0.7)
xlabel = ax0.set_xlabel('lsst_u mock')
ylabel = ax0.set_ylabel('lsst_u recomputed')
xlabel = ax1.set_xlabel('recomputed - mock')

In [ ]:
assert np.allclose(mock['lsst_u'], phot_data['obs_mags'][:, 0], rtol=1e-4)

### Inspect disk and bulge assembly histories

In [ ]:
from diffsky.experimental.disk_bulge_modeling import disk_bulge_kernels as dbk
fbulge_params = dbk.DEFAULT_FBULGE_PARAMS._make((mock['fbulge_tcrit'], mock['fbulge_early'], mock['fbulge_late']))
_res = dbk._bulge_sfh_vmap(mock_info['t_table'], phot_data['sfh_table'], fbulge_params)
# smh, eff_bulge, sfh_bulge, smh_bulge, 
bth = _res[-1]

In [ ]:
vmap_interp = jjit(vmap(jnp.interp, in_axes=(0, None, 0)))
bt = vmap_interp(mock['t_obs'], t_table, bth)

fig, ax = plt.subplots(1, 1)
__=ax.scatter(bt, phot_data['bulge_to_total'],s=1)

In [ ]:
param_collection._fields

In [ ]:
delta_scatter = np.where(
    mock["mc_sfh_type"].reshape((-1, 1)) == 0,
    mock["delta_scatter_q"],
    mock["delta_scatter_ms"],
)

In [ ]:
from diffsky.experimental.dbk_from_mock import _disk_bulge_knot_phot_from_mock

args = (
    mock['redshift_true'], t_obs, mah_params, mock['logmp0'], t_table,
    ssp_data, precomputed_ssp_mag_table, z_phot_table, wave_eff_table, sfh_params,
    param_collection.mzr_params, param_collection.spspop_params, 
    param_collection.scatter_params, param_collection.ssperr_params,
    sim_info.cosmo_params, sim_info.fb, 
    mock['uran_av'], mock['uran_delta'], mock['uran_funo'], 
    delta_scatter, mock["mc_sfh_type"], mock["fknot"]
)
phot_info = _disk_bulge_knot_phot_from_mock(*args)

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.scatter(phot_info['fbulge_early'], mock['fbulge_early'], s=1)

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.scatter(phot_info['obs_mags'][:, 0], mock['lsst_u'], s=1)

In [ ]:
fig, ax = plt.subplots(1, 1)

__=ax.hist(phot_info['obs_mags'][:, 0]-mock['lsst_u'], alpha=0.7)

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.scatter(phot_info['logsm_obs'], mock['logsm_obs'], s=1)

In [ ]:
from diffsky.experimental.disk_bulge_modeling import mc_disk_bulge as mcdb

disk_bulge_history = mcdb.decompose_sfh_into_disk_bulge_sfh(t_table, sfh_table)
disk_bulge_history._fields